In [1]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../py_files')
from sdlnano import BayesianOptimizer_Auto
from calculation import BatchCalculation

In [ ]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [3]:
iteration = BayesianOptimizer_Auto.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 1


# Check the drug

In [4]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [5]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = BayesianOptimizer_Auto.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

[INFO 02-11 09:01:09] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 02-11 09:01:09] ax.modelbridge.transforms.standardize_y: Outcome Complexity is constant, within tolerance.
/Users/zeqing/opt/anaconda3/envs/sdlnano/lib/python3.10/site-packages/botorch/models/utils/assorted.py:268: InputDataWarning: Data (outcome observations) is not standardized (std = tensor([0., 1., 1., 1.], dtype=torch.float64), mean = tensor([ 0.0000e+00, -1.1102e-16, -6.9389e-17,  9.0206e-17],
       dtype=torch.float64)).Please consider scaling the input to zero mean and unit variance.
  check_standardization(Y=train_Y, raise_on_fail=raise_on_fail)
/Users/zeqing/opt/anaconda3/envs/sdlnano/lib/python3.10/site-packages/ax/modelbridge/cross_validation.py:556: RuntimeWarning: divide by zero encountered in scalar divide
  inv_model_std_quality = max_std if ma

1 min


In [ ]:
# process the trails into ratios

unlabeled_trial_processed = BayesianOptimizer_Auto.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.0,16.00,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,23.500000,354.0,3.91,67.0,0.172812,0.026625,0.129250,0.022063,0.038937,0.164062,0.211312,0.053687,0.089500,0.091688,0.398562,0.218688,0.097312,0.285562
std,4.760952,0.0,0.00,0.0,0.081838,0.057912,0.118463,0.018603,0.020664,0.086447,0.060906,0.086695,0.101566,0.116297,0.320572,0.185452,0.139866,0.218460
min,16.000000,354.0,3.91,67.0,0.000000,0.000000,0.000000,0.000000,0.013000,0.000000,0.092000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.750000,354.0,3.91,67.0,0.132250,0.000000,0.000000,0.014000,0.031250,0.117250,0.181500,0.000000,0.000000,0.000000,0.099000,0.036750,0.000000,0.065250
50%,23.500000,354.0,3.91,67.0,0.176000,0.000000,0.154500,0.019500,0.033500,0.141000,0.220500,0.000000,0.044000,0.000000,0.379500,0.192000,0.014000,0.337500
75%,27.250000,354.0,3.91,67.0,0.239750,0.000000,0.212750,0.025250,0.047500,0.228500,0.248000,0.071250,0.182500,0.228500,0.554000,0.372500,0.195750,0.500000
max,31.000000,354.0,3.91,67.0,0.324000,0.165000,0.321000,0.075000,0.099000,0.324000,0.311000,0.246000,0.263000,0.266000,0.972000,0.500000,0.400000,0.541000


In [7]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000,16.000,16.000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,23.500000,0.354,0.391,0.067,0.720835,0.164200,0.533850,0.099306,0.172195,0.667299,0.879088,0.245752,0.411497,0.378595,0.686948,0.495239,0.231246,0.600101
std,4.760952,0.000,0.000,0.000,0.322669,0.361662,0.473890,0.106709,0.125133,0.295453,0.223944,0.402690,0.465583,0.479037,0.448297,0.431730,0.373019,0.466919
min,16.000000,0.354,0.391,0.067,0.000000,0.000000,0.000000,0.000000,0.042051,0.000000,0.335952,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.750000,0.354,0.391,0.067,0.567887,0.000000,0.000000,0.043974,0.135033,0.499222,0.894808,0.000000,0.000000,0.000000,0.227702,0.049735,0.000000,0.071797
50%,23.500000,0.354,0.391,0.067,0.803791,0.000000,0.660477,0.078173,0.153366,0.622923,1.000000,0.000000,0.169503,0.000000,1.000000,0.440470,0.014522,0.950027
75%,27.250000,0.354,0.391,0.067,1.000000,0.000000,1.000000,0.105282,0.177715,1.000000,1.000000,0.297122,1.000000,1.000000,1.000000,0.916304,0.306351,1.000000
max,31.000000,0.354,0.391,0.067,1.000000,1.000000,1.000000,0.453739,0.595994,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
# Convert ratios to volumes

transfer = BatchCalculation.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.0,16.00,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.00000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,23.500000,354.0,3.91,67.0,60.484375,9.318750,45.237500,7.721875,13.628125,57.421875,73.959375,18.790625,31.32500,32.090625,398.562500,218.687500,97.312500,285.562500,0.021875
std,4.760952,0.0,0.00,0.0,28.643380,20.269352,41.461928,6.510964,7.232357,30.256542,21.317162,30.343128,35.54824,40.703805,320.571774,185.452498,139.865516,218.460361,0.413509
min,16.000000,354.0,3.91,67.0,0.000000,0.000000,0.000000,0.000000,4.550000,0.000000,32.200000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.700000
25%,19.750000,354.0,3.91,67.0,46.287500,0.000000,0.000000,4.900000,10.937500,41.037500,63.525000,0.000000,0.00000,0.000000,99.000000,36.750000,0.000000,65.250000,-0.350000
50%,23.500000,354.0,3.91,67.0,61.600000,0.000000,54.075000,6.825000,11.725000,49.350000,77.175000,0.000000,15.40000,0.000000,379.500000,192.000000,14.000000,337.500000,0.000000
75%,27.250000,354.0,3.91,67.0,83.912500,0.000000,74.462500,8.837500,16.625000,79.975000,86.800000,24.937500,63.87500,79.975000,554.000000,372.500000,195.750000,500.000000,0.350000
max,31.000000,354.0,3.91,67.0,113.400000,57.750000,112.350000,26.250000,34.650000,113.400000,108.850000,86.100000,92.05000,93.100000,972.000000,500.000000,400.000000,541.000000,0.700000


In [9]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug       1.0
SL_1       0.1
SL_2       0.7
SL_3       0.1
LL_1       0.2
LL_2       0.9
LL_3       1.2
P_1        0.3
P_2        0.5
P_3        0.5
S_1        6.4
S_2        3.5
S_3        1.6
Water      4.6
Solvent    0.0
dtype: float64

In [10]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [11]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.drop(['Drug_MW', 'Drug_LogP', 'Drug_TPSA'], axis=1).to_excel(transfer_path, index = False)

[INFO 02-11 09:02:55] ax.service.ax_client: Saved JSON-serialized state of optimization to `optimizer/optimizer_design_1.json`.


# Generate new protocol

In [12]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_.py'


In [13]:
# Update the script with Excel data
updated_script = BatchCalculation.update_transfer_script(template_protocol, transfer_path, output_script_path) 


In [14]:
updated_script

'protocol/iteration_1_OT_2_protocol_.py'

# Perform experiment

# Results

In [16]:
results = BatchCalculation.parameters_and_complexity(iteration)
results

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,16,354,3.91,67,0.135,0.000,0.000,0.020,0.032,0.093,0.225,0.046,0.225,0.225,0.741,0.049,0.210,0.000,10,0
1,17,354,3.91,67,0.243,0.000,0.187,0.029,0.042,0.148,0.254,0.000,0.098,0.000,0.333,0.183,0.000,0.484,8,0
2,18,354,3.91,67,0.187,0.117,0.000,0.019,0.032,0.119,0.187,0.000,0.187,0.154,0.683,0.153,0.040,0.124,10,0
3,19,354,3.91,67,0.000,0.000,0.000,0.000,0.014,0.000,0.246,0.246,0.246,0.246,0.400,0.201,0.400,0.000,8,0
4,20,354,3.91,67,0.242,0.000,0.242,0.024,0.034,0.242,0.217,0.000,0.000,0.000,0.132,0.434,0.000,0.434,7,0
5,21,354,3.91,67,0.139,0.000,0.224,0.018,0.036,0.134,0.224,0.224,0.000,0.000,0.500,0.000,0.000,0.500,7,0
6,22,354,3.91,67,0.124,0.000,0.321,0.015,0.018,0.225,0.296,0.000,0.000,0.000,0.000,0.435,0.059,0.507,7,0
7,23,354,3.91,67,0.239,0.000,0.000,0.016,0.029,0.239,0.212,0.000,0.026,0.239,0.333,0.333,0.333,0.000,9,0
8,24,354,3.91,67,0.209,0.000,0.209,0.033,0.033,0.127,0.209,0.000,0.181,0.000,0.000,0.459,0.000,0.541,7,0
9,25,354,3.91,67,0.242,0.000,0.071,0.021,0.047,0.321,0.224,0.000,0.000,0.074,0.359,0.319,0.000,0.323,8,0


In [17]:
size_raw = BatchCalculation.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Good,A1,231.980508,0.374270
1,Good,A2,236.372324,0.329095
2,Good,A3,245.335137,0.293648
3,Caution,A4,312.360093,0.570796
4,Good,A5,282.510319,0.461832
5,Caution,A6,295.441290,0.481025
6,Good,B1,258.033612,0.364754
7,Good,B2,243.106386,0.320841
8,Good,B3,242.000500,0.303758
9,Caution,B4,176.525646,0.309924


In [18]:
size_processed = BatchCalculation.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,237.895990,6.806446,0.332338,0.040409,1
1,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,Formulation 3,247.713499,8.954568,0.329785,0.031466,1
3,Formulation 4,1000.000000,0.000000,1.000000,0.000000,0
4,Formulation 5,1000.000000,0.000000,1.000000,0.000000,0
5,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0
6,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0
7,Formulation 8,285.015102,12.004342,0.315194,0.071264,1
8,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0
9,Formulation 10,1000.000000,0.000000,1.000000,0.000000,0


In [19]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,16,354,3.91,67,0.135,0.000,0.000,0.020,0.032,0.093,...,0.210,0.000,10,0,Formulation 1,237.895990,6.806446,0.332338,0.040409,1
1,17,354,3.91,67,0.243,0.000,0.187,0.029,0.042,0.148,...,0.000,0.484,8,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,18,354,3.91,67,0.187,0.117,0.000,0.019,0.032,0.119,...,0.040,0.124,10,0,Formulation 3,247.713499,8.954568,0.329785,0.031466,1
3,19,354,3.91,67,0.000,0.000,0.000,0.000,0.014,0.000,...,0.400,0.000,8,0,Formulation 4,1000.000000,0.000000,1.000000,0.000000,0
4,20,354,3.91,67,0.242,0.000,0.242,0.024,0.034,0.242,...,0.000,0.434,7,0,Formulation 5,1000.000000,0.000000,1.000000,0.000000,0
5,21,354,3.91,67,0.139,0.000,0.224,0.018,0.036,0.134,...,0.000,0.500,7,0,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0
6,22,354,3.91,67,0.124,0.000,0.321,0.015,0.018,0.225,...,0.059,0.507,7,0,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0
7,23,354,3.91,67,0.239,0.000,0.000,0.016,0.029,0.239,...,0.333,0.000,9,0,Formulation 8,285.015102,12.004342,0.315194,0.071264,1
8,24,354,3.91,67,0.209,0.000,0.209,0.033,0.033,0.127,...,0.000,0.541,7,0,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0
9,25,354,3.91,67,0.242,0.000,0.071,0.021,0.047,0.321,...,0.000,0.323,8,0,Formulation 10,1000.000000,0.000000,1.000000,0.000000,0


In [20]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,16,354,3.91,67,0.135,0.000,0.000,0.020,0.032,0.093,...,10,0,Formulation 1,237.895990,6.806446,0.332338,0.040409,1,270.0,0
1,17,354,3.91,67,0.243,0.000,0.187,0.029,0.042,0.148,...,8,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
2,18,354,3.91,67,0.187,0.117,0.000,0.019,0.032,0.119,...,10,0,Formulation 3,247.713499,8.954568,0.329785,0.031466,1,374.0,0
3,19,354,3.91,67,0.000,0.000,0.000,0.000,0.014,0.000,...,8,0,Formulation 4,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
4,20,354,3.91,67,0.242,0.000,0.242,0.024,0.034,0.242,...,7,0,Formulation 5,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
5,21,354,3.91,67,0.139,0.000,0.224,0.018,0.036,0.134,...,7,0,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
6,22,354,3.91,67,0.124,0.000,0.321,0.015,0.018,0.225,...,7,0,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
7,23,354,3.91,67,0.239,0.000,0.000,0.016,0.029,0.239,...,9,0,Formulation 8,285.015102,12.004342,0.315194,0.071264,1,478.0,0
8,24,354,3.91,67,0.209,0.000,0.209,0.033,0.033,0.127,...,7,0,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
9,25,354,3.91,67,0.242,0.000,0.071,0.021,0.047,0.321,...,8,0,Formulation 10,1000.000000,0.000000,1.000000,0.000000,0,0.0,0


In [21]:
results_normalized = BayesianOptimizer_Auto.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.237896,0.006806,0.135,0.0,0.332338,0.040409,0.833333
1,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.666667
2,1,0.247713,0.008955,0.187,0.0,0.329785,0.031466,0.833333
3,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.666667
4,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.583333
5,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.583333
6,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.583333
7,1,0.285015,0.012004,0.239,0.0,0.315194,0.071264,0.750000
8,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.583333
9,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.666667


In [22]:
results_normalized = BayesianOptimizer_Auto.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.237896,0.006806,0.135,0.0,0.332338,0.040409,0.833333
1,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.666667
2,1,0.247713,0.008955,0.187,0.0,0.329785,0.031466,0.833333
3,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.666667
4,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.583333
5,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.583333
6,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.583333
7,1,0.285015,0.012004,0.239,0.0,0.315194,0.071264,0.750000
8,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.583333
9,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.666667


In [23]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [24]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [25]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,16,354,3.91,67,0.135,0.000,0.000,0.020,0.032,0.093,...,0.833333,0,Formulation 1,0.237896,0.006806,0.332338,0.040409,1,0.135,0
1,17,354,3.91,67,0.243,0.000,0.187,0.029,0.042,0.148,...,0.666667,0,Formulation 2,1.000000,0.000000,1.000000,0.000000,0,0.000,0
2,18,354,3.91,67,0.187,0.117,0.000,0.019,0.032,0.119,...,0.833333,0,Formulation 3,0.247713,0.008955,0.329785,0.031466,1,0.187,0
3,19,354,3.91,67,0.000,0.000,0.000,0.000,0.014,0.000,...,0.666667,0,Formulation 4,1.000000,0.000000,1.000000,0.000000,0,0.000,0
4,20,354,3.91,67,0.242,0.000,0.242,0.024,0.034,0.242,...,0.583333,0,Formulation 5,1.000000,0.000000,1.000000,0.000000,0,0.000,0


In [26]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


[INFO 02-11 10:57:38] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.


In [27]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [28]:
loaded_ax_client = BayesianOptimizer_Auto.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 02-11 10:57:39] ax.service.ax_client: Completed trial 16 with data: {'Solu': (0.135, 0), 'Size': (0.237896, 0.006806), 'PDI': (0.332338, 0.040409), 'Complexity': (0.833333, 0)}.
[INFO 02-11 10:57:39] ax.service.ax_client: Completed trial 17 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0), 'Complexity': (0.666667, 0)}.
[INFO 02-11 10:57:39] ax.service.ax_client: Completed trial 18 with data: {'Solu': (0.187, 0), 'Size': (0.247713, 0.008955), 'PDI': (0.329785, 0.031466), 'Complexity': (0.833333, 0)}.
[INFO 02-11 10:57:39] ax.service.ax_client: Completed trial 19 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0), 'Complexity': (0.666667, 0)}.
[INFO 02-11 10:57:39] ax.service.ax_client: Completed trial 20 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0), 'Complexity': (0.583333, 0)}.
[INFO 02-11 10:57:39] ax.service.ax_client: Completed trial 21 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0), 'Complexity': 